<!-- #### 节点

对于每一个节点，应当至少由以下元件部分组成：

- 切换器A `switchA`

  不可靠元件，存在失效风险。
  - 工作状态 `[0,1,2,3]`
    - 正常A0
    - 故障A1：掷刀无法脱离触点1
    - 故障A2：掷刀无法脱离触点2
    - 故障A3：掷刀无法接合任意触点
  - 掷刀位置 `[1,2,3]`
    - 触点1
    - 触点2
    - 断开

- 切换器B `switchB`

  不可靠元件，存在失效风险。
  - 工作状态 `[0,1,2]`
    - 正常B0
    - 故障B1：掷刀无法脱离触点
    - 故障B2：掷刀无法接合触点
  - 掷刀位置 `[1,2]`
    - 接合
    - 断开

- 节点性能状态 `[0,1,2,3,4,5]`

  节点性能状态完全由切换器状态决定，参考指导书表格
  - gN0: (gPF perfectly functioning)
  - gN1: (gSO slave only)
  - gN2: (gDM disable/master)
  - gN3: (gMO master only) 只能作为主节点，否则就会阻塞总线
  - gN4: (gDN disable node)
  - gN5: (gFB failed bus) 节点总是阻塞总线 -->


<!-- ![](node-state.png) -->


<!-- 注意，这里，我先暂时使用全局变量，之后可以看情况用结构体来封装简化代码

除此以外，使用局部变量能使寄存器得到充分的使用，提高效率 -->


<!-- 经过仔细的思考，我重新认为，本实验的最小控制单位是元件的状态，而不是掷刀的位置，这里的抽象是为了简化考虑过程，所以想复杂了。

那么对于是否是主节点就不应该使用掷刀位置等的判断，而是从一开始就选定，比如列一个flag，节点的重选过程应当只下到“状态”这一级，与掷刀位置无关。 -->


节点角色转换公式

$$
\{G_{role}^{(m)}\}=F_{G}(\{G_{role}^{(m-1)}\},\{G_{N}^{(m)}\},[random])
$$


卡住了，如何判断总线时钟失效，在什么地方重选主节点，怎么确定当前主节点的编号？


In [42]:
using Statistics
using Printf
using LinearAlgebra

# parameters
## system 
# NUM_SYSTEM = 10_0000
NUM_SYSTEM = 1000
NUM_NODE = 10               # this should also be optimized later
TIME_STEP = 1               # 1 hour
LIFE_LIMIT = 20_0000
STATE_NUM_NODE = 6
k = 3
## switch A
λA = 1 / 5.90e4             # hour
PA0 = exp(-λA * TIME_STEP)
PEA1 = 0.20 * (1 - PA0)
PEA2 = 0.15 * (1 - PA0)
PEA3 = 0.65 * (1 - PA0)

## switch B
λB = 1 / 2.20e5             # hour
PB0 = exp(-λB * TIME_STEP)
PEB1 = 0.45 * (1 - PB0)
PEB2 = 0.55 * (1 - PB0)

# println("PA0=$PA0\tPB0=$PB0")
# println("PEA1=$PEA1\tPEA2=$PEA2\tPEA3=$PEA3")
# println("PEB1=$PEB1\tPEB2=$PEB2")

2.4999943182002493e-6

In [43]:
# variables definition for each simulation
gA = zeros(Integer, NUM_NODE)  # a better choice here should be @enum rather than Integer
gB = zeros(Integer, NUM_NODE)  # a better choice here should be @enum rather than Integer
posA = zeros(Integer, NUM_NODE)  # a better choice here should be @enum rather than Integer
posB = zeros(Integer, NUM_NODE)  # a better choice here should be @enum rather than Integer

gN = zeros(Integer, NUM_NODE)
system_life = zeros(Integer, NUM_SYSTEM)
# Gsys = 0 # valid only during the operation of current system
# QPF = 0  # QN0 = QPF
# QSO = 0  # QN1 = QSO
# QDM = 0  # QN2 = QDM
# QMO = 0  # QN3 = QMO
# QDN = 0  # QN4 = QDN
# QFB = 0  # QN5 = QFB

master_counter = 0


0

In [44]:
function estimate_switch_state!(gA, gB) # 考虑到节点的状态完全由切换器的状态决定，又回想起来前面写过了什么，然后把最后一块拼图拼上了
    # 各switch有一定概率正常工作，一定概率出现异常
    sampleA = rand(NUM_NODE)
    sampleB = rand(NUM_NODE)
    for i = 1:NUM_NODE
        if gA[i] != 0 || sampleA[i] < PA0 #当前切换器已经坏了，不需要计算||仍然正常工作
            continue
        else
            tol = sampleA[i] - PA0
            if tol < PEA1
                gA[i] = 1
            elseif (tol < PEA1 + PEA2)
                gA[i] = 2
            else
                gA[i] = 3
            end
            # gA[i] = tol < PEA1 ? 1 : tol < PEA1 + PEA2 ? 2 : 3
        end
        if gB[i] != 0 || sampleB[i] < PB0
            continue
        else
            gB[i] = sampleB[i] < PB0 + PEB1 ? 1 : 2
        end
        # Todo: 以上代码应该可以改成位运算，现在为了直观，不做优化
    end
end

estimate_switch_state! (generic function with 1 method)

In [45]:
function estimate_node_state!(gA, gB, gN)
    # 这之前已经修改过switch state了
    for i = 1:NUM_NODE
        if gA[i] == 0
            # 为了好的观感稍稍的降低了效率，相当于if并列而没有elseif
            gB[i] == 0 && (gN[i] = 0) # dont change masterflag
            gB[i] == 1 && (gN[i] = 3)
            gB[i] == 2 && (gN[i] = 1) #, (master_flag[i] = false))
        elseif gA[i] == 1
            gB[i] == 0 && (gN[i] = 1) #, master_flag[i]=false)
            gB[i] == 1 && (gN[i] = 5)
            gB[i] == 2 && (gN[i] = 1) #, master_flag[i]=false)
        elseif gA[i] == 2
            gB[i] == 0 && (gN[i] = 2)
            gB[i] == 1 && (gN[i] = 3)
            gB[i] == 2 && (gN[i] = 4)
        elseif gA[i] == 3
            gB[i] == 0 && (gN[i] = 4)
            gB[i] == 1 && (gN[i] = 4)
            gB[i] == 2 && (gN[i] = 4)
        end
    end
end

estimate_node_state! (generic function with 1 method)

In [46]:
function initialize!(gA, gB, gN, master_flag, master_node)
    # 最开始，不妨设第1个节点为主节点，其它节点为从节点
    # Todo: 这是可以修改的，也可以用随机算法，但意义不大
    fill!(gA, 0)  # 状态全清0，代表正常
    fill!(gB, 0)

    # 设置第1个节点为主节点
    fill!(master_flag, 0)
    master_node = 1
    master_flag[master_node] = 1      #! 这部分是写到后面才想起来加上的
    # master_counter = 1      #! 这部分是写到后面才想起来加上的
    # 计算节点状态
    estimate_node_state!(gA, gB, gN)
end

initialize! (generic function with 1 method)

In [47]:
function count_master_num(master_flag)
    num = sum(master_flag)
end

count_master_num (generic function with 1 method)

In [48]:
function update_node_state!(gA, gB, gN, master_flag,master_node) # 2 当准备开始写了，发现有需要的前置条件，
    estimate_switch_state!(gA, gB)
    estimate_node_state!(gA, gB, gN)
    # 在这两步骤之后，应该是已经可以知道系统是否能正常运作了
    # 问题在于，用什么来衡量系统时钟出错了
    reselect_master!(gA, gB, gN, master_flag,master_node)
end

update_node_state! (generic function with 1 method)

这里节点重选的地方还没能想清楚

从最初始的地方开始，是主节点在 1 号，然后剩余节点都是从节点

在 update_node_state 之后，由于对于 switch 状态的重新估计，导致了 node 的状态可能发生改变。

这时候，需要对接下来可能发生变化的系统状态进行讨论，

我们要进行节点重选，是因为更新节点状态之后可能会导致系统出现异常，那么，这里能不能套用后面计算系统状态的结论？如果不行的话，这个分析过程将会变得相当复杂，等价于分析后面系统状态变化的工作量。

重选节点的算法本身很简单，但是如何选择可以用于重选的节点？这个限制条件要怎么加？

现在我已经有了什么：所有节点的状态，以及一个最初设定的主节点 1

现在遇到的困难是，如果主节点不是 1 了，我要如何找出所选的主节点，以及当存在了多个主节点的时候，要如何选择去除多余的主节点。

那么，现在为了简化问题，我先强行要求只能有一个主节点，否则系统异常，查找可以去除的主节点，并完成节点重选

我认为，主节点可以从 MO 里面添加，但是不能从节点更新就开始就添加会比较好处理
即，在节点更新函数里面，可以使主节点个数减少到 0，但不能主动设置主节点


In [49]:
#! 这部分应该是整个系统里最麻烦的地方
# 注意，从描述中可知，如果系统处于正常工作状态，哪怕有异常节点，也不会触发主
# 节点重选，重选主节点一定发生在系统处于异常时，但问题在于，这个时候不应该认
# 为系统失效，所以不能完全应用Gsys的状态（否则就要统计两次Gsys的状态）
# 除此以外，这里的“戒备状态”事实上并不是真的时间，而是随机产生一组随机数，选
# 择最小数下标的作为重选的主节点很容易实现： 
# _, new_master_index = findmin(rand(NUM_NODE))
# 这里再根据具体的要求改改就可以了
# 问题在于，如何判断总线时钟失效，以及如果当前有多个主节点怎么办？
# 先停这里，写update_node_state
# if clock_invalid
#     master_flag
# end
function ok_for_master(gNi)
    gNi == 0 && return true
    gNi == 1 && return false
    gNi == 2 && return true
    gNi == 3 && return true # MO
    gNi == 4 && return false
    gNi == 5 && return false
end
function reselect_master!(gA, gB, gN, master_flag, master_node)  # 1
    # index = findfirst(master_flag)
    if !ok_for_master(gN[master_node])
        # 说明主节点坏掉了
        alert_mintime = 1.0 # rand() [0.0,1)
        mintime_index = 0
        alert_count = rand(NUM_NODE)
        for i = 1:NUM_NODE
            if ok_for_master(gN[i])
                tmp = alert_mintime
                alert_mintime = min(alert_mintime, alert_count[i])
                mintime_index = tmp != alert_mintime ? i : mintime_index
            end
        end
        master_node = mintime_index
        master_flag[master_node] = 1
    end
    # else # 把这个else删掉后，就可以把两种情况放在一起用了
    # 说明原来的主节点还可以用，但信号出现异常，这个时候要么是MO，要么是FB
    # 我们只需要讨论一种情况，如果系统有一个MO出现
    cnt = 0
    idx = 0
    for i = 1:NUM_NODE
        gN[i] == 3 && (cnt=cnt + 1, idx=i)
    end
    if cnt == 1
        master_flag[idx] = 1
        master_flag[master_node] = 0
        master_node = idx
    elseif cnt >= 2
        master_flag[master_node] = 0
        master_node = 0 # 这里随意设置，因为系统必然失效
    elseif cnt == 0 && (5 in gN)
        master_flag[master_node] = 0
        master_node = 0 # 这里随意设置，因为系统必然失效
    else
        # 也就是说现在信号是好的，不需要重选
    end
    # end
end

reselect_master! (generic function with 1 method)

我发现文本的解释好像有点问题，或者是我的理解有误

> 有且仅有一个节点处于 gMO（注：按前文描述的系统工作机制，该节点必然担当主节点，虽然因为随机因素，过程可能曲折）

按理说，MO 节点应当是始终处于主模式的，无法退出到从模式，所以必然直接占据了主模式的信号，从而不需要重选，重选的情况应该只会发生在，主模式因故失效的情况下


In [50]:
function simulate!(life, gA, gB, gN)
    master_flag = zeros(Bool, NUM_NODE)
    master_node = 1

    initialize!(gA, gB, gN, master_flag, master_node)
    Gsys = 0  # 初始化后的系统可以正常工作
    state_system = false
    life_counter = 0
    while !state_system && life_counter < LIFE_LIMIT
        # 更新各节点的状态
        update_node_state!(gA, gB, gN, master_flag, master_node)
        # 计算系统的状态，我是先写的这一部分，没有按着顺序从头写到尾
        QPF = QSO = QDM = QMO = QDN = QFB = 0 # Todo: shoud be initialized earlier 
        for elem in gN
            elem == 0 && (QPF += 1)
            elem == 1 && (QSO += 1)
            elem == 2 && (QDM += 1)
            elem == 3 && (QMO += 1)
            elem == 4 && (QDN += 1)
            elem == 5 && (QFB += 1)
        end
        # 这之前 Gsys=0，初始化0不是一个好的选择，而且以下判断可能需要放到前面去
        if QFB >= 1 || QMO >= 2 || QPF + QMO + QDM == 0 || (QPF + QSO + 1((QMO + QDM) > 0)) < k
            Gsys = 1
        elseif QFB == 0 && ((QMO == 1 && QPF + QSO >= k - 1) || (QMO == 0 && QPF >= 1 && QPF + QSO >= k || QMO == 0 && QPF == 0 && QDM >= 1 && QSO >= k - 1))
            Gsys = 2
            # 这里的条件文本没有给清楚，但大致能猜到C5 && (C6 || C7)
        elseif QFB + QMO == 0 && (QPF >= 1 && QPF + QSO == k - 1 && QDM >= 1)
            # Todo: 明确节点的选择是在哪里发生的
            # Todo: 明确指导书中写在这里的条件概率有什么用，我是在做仿真而不是理论分析
            if gN[master_node] == 2
                Gsys = 3  # if one of gDM is selected as master
            elseif gN[master_node] == 0
                Gsys = 4  # if one of gPF is selected as master, fail to satisfy valid node limit k
            end
        end
        if Gsys == 2 || Gsys == 3
            life_counter += 1
        else
            state_system = true
        end
        if life_counter % 100 == 99
            println("life_counter: $life_counter")
        end
    end
    life = life_counter
end

simulate! (generic function with 1 method)

In [51]:
for i = 1:NUM_SYSTEM
    simulate!(system_life[i], gA, gB, gN)
end

mean(system_life)

life_counter: 99
life_counter: 199
life_counter: 299
life_counter: 399
life_counter: 499
life_counter: 599
life_counter: 699
life_counter: 799
life_counter: 899
life_counter: 999
life_counter: 1099
life_counter: 1199
life_counter: 1299
life_counter: 1399
life_counter: 1499
life_counter: 1599
life_counter: 1699
life_counter: 1799
life_counter: 1899
life_counter: 1999
life_counter: 2099
life_counter: 2199
life_counter: 2299
life_counter: 2399
life_counter: 2499
life_counter: 2599
life_counter: 2699
life_counter: 2799
life_counter: 2899
life_counter: 2999
life_counter: 3099
life_counter: 3199
life_counter: 3299
life_counter: 3399
life_counter: 3499
life_counter: 3599
life_counter: 3699
life_counter: 3799
life_counter: 3899
life_counter: 3999
life_counter: 4099
life_counter: 4199
life_counter: 4299
life_counter: 4399
life_counter: 4499
life_counter: 4599
life_counter: 4699
life_counter: 4799
life_counter: 4899
life_counter: 4999
life_counter: 5099
life_counter: 5199
life_counter: 5299
life

Excessive output truncated after 524306 bytes.

life_counter: 97299
life_counter: 97399
life_counter: 97499
life_counter: 97599
life_counter: 97699
life_counter: 97799
life_counter: 97899
life_counter: 97999
life_counter: 98099
life_counter: 98199
life_counter: 98299
life_counter: 98399
life_counter: 98499
life_counter: 98599
life_counter: 98699
life_counter: 98799
life_counter: 98899
life_counter: 98999
life_counter: 99099
life_counter: 99199
life_counter: 99299
life_counter: 99399
life_counter: 99499
life_counter: 99599
life_counter: 99699
life_counter: 99799
life_counter: 99899
life_counter: 99999
life_counter: 100099
life_counter: 100199
life_counter: 100299
life_counter: 100399
life_counter: 100499
life_counter: 100599
life_counter: 100699
life_counter: 100799
life_counter: 100899
life_counter: 100999
life_counter: 101099
life_counter: 101199
life_counter: 101299
life_counter: 101399
life_counter: 101499
life_counter: 101599
life_counter: 101699
life_counter: 101799
life_counter: 101899
life_counter: 101999
life_counter: 102099

LoadError: BoundsError: attempt to access 10-element Vector{Bool} at index [0]